In [1]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

In [3]:
%pwd

'/home/ananta/mds/block6/dsci-525'

In [4]:
%cd /home/ananta/mds/block6/dsci-525/

/home/ananta/mds/block6/dsci-525


In [5]:
article_id = 14096681
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "figshare_rainfall/"

In [6]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://

In [7]:
%%time
files_to_dl = ["data.zip"]
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

CPU times: user 2.11 s, sys: 2.56 s, total: 4.67 s
Wall time: 55.6 s


In [8]:
%%time
with zipfile.ZipFile(os.path.join(output_directory, "data.zip"), 'r') as f:
    f.extractall(output_directory)

CPU times: user 15.1 s, sys: 3.71 s, total: 18.8 s
Wall time: 29.1 s


In [9]:
### just listing to get an idea how individual file looks like 
sample_df = pd.read_csv(output_directory + "ACCESS-CM2_daily_rainfall_NSW.csv")
sample_df.columns

Index(['time', 'lat_min', 'lat_max', 'lon_min', 'lon_max', 'rain (mm/day)'], dtype='object')

In [ ]:
### making sure that all files have the same colums
files = glob.glob(os.path.join(output_directory, "*.csv"))
use_cols = list(sample_df.columns)
for file in files:
    try:
        pd.read_csv(file, index_col=0, usecols=use_cols)
    except:
        df = pd.read_csv(file, index_col=0)
        print(f"{file} does not have all columns. {file} only has {df.columns}.")
        continue

figshare_rainfall/observed_daily_rainfall_SYD.csv does not have all columns. figshare_rainfall/observed_daily_rainfall_SYD.csv only has Index(['rain (mm/day)'], dtype='object').


In [ ]:
%%time
%memit
# "figshare_rainfall/observed_daily_rainfall_SYD.csv" is missing 'lat_min', 'lat_max', 'lon_min', 'lon_max' columns
files = glob.glob(os.path.join(output_directory, "*_NSW.csv"))

# combining using pandas method
df = pd.concat((pd.read_csv(file, index_col=0, usecols=use_cols)
                #TODO: Find a better way to do this
                .assign(model = file.replace("_daily_rainfall_NSW.csv", "").replace("figshare_rainfall/", ""))
                for file in files)
              )
df.to_csv(os.path.join(output_directory, "combined_data.csv"))